In [ ]:
from _util import *

import _RL.FQI as FQI
import _RL.my_gym as my_gym
import _Ohio_Simulator as Ohio
import _analyze as analyze
import _RL.DQN as DQN
import _cartpole as cartpole
from coinDice import run_neural_coin_dice as run_coin
from coinDice import converter as converter

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

reload(converter)
reload(run_coin)

1. we can either store the data generated from TR, or re-generate it 

## N20, T = 200, tau = 0.8, algae_alpha = 2.5

In [ ]:
unique_setting = "ohio_tau_Qs_coindice"
N = 20
tau = 0.8
###################################################################################################################
n_gpu = 16
rep = 200

tp_path = "target_policies/1003_gamma95_sd10/1003_gamma95_sd10_50"
########################################################################
sd_G = 10
T = 200
gamma = .95
################################################################################################################################################ 
###################################################### Default Settings ######################################################
################################################################################################################################################
setting = EST()[7:9] + EST()[10:13]+ "_"  + unique_setting + "_" + "N" + str(N) + "_" + "T" + str(T) + "_" + "sd" + str(sd_G)  + "_"  + "tau" + "_" + str(tau)
printR("setting: {}".format(setting))
################################################################################################################################################

####################################
FQI_paras = {"hiddens" : [64, 64], "gamma" : gamma}
pi1 = FQI.FQI(gpu_number = 0, **FQI_paras)
pi1.model.load_weights(tp_path)
pi_behva = my_gym.softmax_policy(pi1, tau)
####################################
ohio = Ohio.OhioSimulator(sd_G = sd_G, T = T, N = N, behav = pi_behva)
V_true = ohio.eval_policy(pi = pi1, gamma = gamma, N = 10000)
init_S = ohio.simu_init_S(seed = 0, N = 1000)
ohio.eval_policy(pi = pi_behva, gamma = gamma, N = 1000)
trajs_train_resp = Ohio.OhioSimulator(sd_G = sd_G, T = T, N = N * 500, behav = pi_behva).simu_one_seed(10000)
trajs_train_resp = [trajs_train_resp[(i * N):((i + 1) * N)] for i in range(rep)]

#########################################################################################################################
#########################################################################################################################
#########################################################################################################################
alpha_CI = 0.9
###########
lr = 0.005
batch_size = 32
algae_alpha = 2.5
ray.shutdown()
a = now()
res_all = []
for i in range(rep // n_gpu):
    @ray.remote(num_gpus=1)
    def one_seed1(seed):
        gpu_idx = seed % n_gpu
        pi1 = FQI.FQI(gpu_number = gpu_idx, **FQI_paras)
        pi1.model.load_weights(tp_path)
        dataset = converter.convertDataset(trajs_train_resp[seed]
                                      , min_A = 0, max_A = 1
                                       , min_S = -5, max_S = 5)
        CI = run_coin.coindice(dataset = dataset, pi1 = pi1, N = N, T = T, gamma = gamma
                      , hidden_dims = (256, 256), gpu_number = gpu_idx
                 , nu_learning_rate = lr, zeta_learning_rate = lr, weight_learning_rate = lr ### ????
                 , nu_regularizer = 1, zeta_regularizer = 1 # NOT SENSITIVE
                 , num_steps = 2000, batch_size = batch_size, alpha_CI = alpha_CI
                 , print_loss = 0, print_freq = 50, print_prog = 0
                 , primal_form = True, f_exponent = 1.5, algae_alpha = algae_alpha)
        return CI
    ray.init()
    ###########
    futures = [one_seed1.remote(j) for j in range(n_gpu * i, n_gpu * (i + 1))]
    res = ray.get(futures)
    ray.shutdown()
    res_all.append(res)
    arr_res_all = np.vstack(res_all)
    coverage = (arr_res_all[:, 0] < -V_true) * (arr_res_all[:, 1] > -V_true)
    mean_width = abs(np.mean(arr_res_all[:, 1] - arr_res_all[:, 0]))
    printR("time cost for {}-th rep = {:.1f} mins with coverage = {} and mean_width = {}".format(i, (now() - a) / 60
                                                                                                 , sum(coverage) / len(coverage)
                                                                                                , mean_width))
res_this_setting = {"CIs" : arr_res_all
                   , "coverage" : coverage
                   , "mean_width" : mean_width}
setting = "ohio_" + "cartpole_" + "alpha_" + str(alpha_CI) + "N_" + str(N) + "tau_" + str(tau) 
setting = setting + "algae_alpha_" + str(algae_alpha) + "lr_" + str(lr)  + "bs_" + str(batch_size)
path = "res/" + setting
printR(path)
dump(res_this_setting, path)
#########################################################################################################################
#########################################################################################################################
#########################################################################################################################

alpha_CI = 0.95
#########################################################################################################################
lr = 0.005
batch_size = 32
algae_alpha = 2.5
ray.shutdown()
a = now()
res_all = []
for i in range(rep // n_gpu):
    @ray.remote(num_gpus=1)
    def one_seed1(seed):
        gpu_idx = seed % n_gpu
        pi1 = FQI.FQI(gpu_number = gpu_idx, **FQI_paras)
        pi1.model.load_weights(tp_path)
        dataset = converter.convertDataset(trajs_train_resp[seed]
                                      , min_A = 0, max_A = 1
                                       , min_S = -5, max_S = 5)
        CI = run_coin.coindice(dataset = dataset, pi1 = pi1, N = N, T = T, gamma = gamma
                      , hidden_dims = (256, 256), gpu_number = gpu_idx
                 , nu_learning_rate = lr, zeta_learning_rate = lr, weight_learning_rate = lr ### ????
                 , nu_regularizer = 1, zeta_regularizer = 1 # NOT SENSITIVE
                 , num_steps = 2000, batch_size = batch_size, alpha_CI = alpha_CI
                 , print_loss = 0, print_freq = 50, print_prog = 0
                 , primal_form = True, f_exponent = 1.5, algae_alpha = algae_alpha)
        return CI
    ray.init()
    ###########
    futures = [one_seed1.remote(j) for j in range(n_gpu * i, n_gpu * (i + 1))]
    res = ray.get(futures)
    ray.shutdown()
    res_all.append(res)
    arr_res_all = np.vstack(res_all)
    coverage = (arr_res_all[:, 0] < -V_true) * (arr_res_all[:, 1] > -V_true)
    mean_width = abs(np.mean(arr_res_all[:, 1] - arr_res_all[:, 0]))
    printR("time cost for {}-th rep = {:.1f} mins with coverage = {} and mean_width = {}".format(i, (now() - a) / 60
                                                                                                 , sum(coverage) / len(coverage)
                                                                                                , mean_width))
res_this_setting = {"CIs" : arr_res_all
                   , "coverage" : coverage
                   , "mean_width" : mean_width}
setting = "ohio_" + "cartpole_" + "alpha_" + str(alpha_CI) + "N_" + str(N) + "tau_" + str(tau) 
setting = setting + "algae_alpha_" + str(algae_alpha) + "lr_" + str(lr)  + "bs_" + str(batch_size)
path = "res/" + setting
printR(path)
dump(res_this_setting, path)

setting: 1106_ohio_tau_Qs_coindice_N20_T200_sd10_tau_0.8
True Value: mean = -47.69 with std = 0.19
True Value: mean = -49.64 with std = 0.62


2020-11-06 05:54:49,975	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8266
2020-11-06 05:55:29,473	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 257455839 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2020-11-06 05:55:43,263	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 05:55:43,264	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 05:55:43,265	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 05:55:43,265	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 05

time cost for 0-th rep = 4.4 mins with coverage = 0.3125 and mean_width = 2.9485671520233154


2020-11-06 05:59:16,373	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-06 05:59:58,942	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 257455839 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2020-11-06 06:00:12,746	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 06:00:12,748	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 06:00:12,749	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 06:00:12,750	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 06

time cost for 1-th rep = 8.8 mins with coverage = 0.3125 and mean_width = 2.906646251678467


...

time cost for 9-th rep = 42.3 mins with coverage = 0.3 and mean_width = 3.481968641281128


2020-11-06 07:26:56,499	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-06 07:27:37,154	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 257455839 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2020-11-06 07:27:51,048	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 07:27:51,049	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 07:27:51,050	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 07:27:51,051	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 07

time cost for 10-th rep = 46.2 mins with coverage = 0.3125 and mean_width = 3.4309678077697754


2020-11-06 07:30:51,603	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-11-06 07:31:33,882	WARNING worker.py:1072 -- Warning: The remote function __main__.one_seed1 has size 257455839 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2020-11-06 07:31:47,600	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 07:31:47,601	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 07:31:47,602	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 07:31:47,602	WARNING worker.py:1072 -- This worker was asked to execute a function that it does not have registered. You may have to restart Ray.
2020-11-06 07

time cost for 11-th rep = 50.6 mins with coverage = 0.3072916666666667 and mean_width = 3.4555938243865967
res/ohio_cartpole_alpha_0.95N_20tau_0.8algae_alpha_2.5lr_0.005bs_32
